In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# --- GLOBAL DRIVE PATH ---
# Make sure to create this folder in your Google Drive first!
DRIVE_ROOT_PATH = "/content/drive/My Drive/Colab Notebooks/bpta/stp"

# --- IMPORTS ---
import heapq
import itertools
import copy
import random
import time
import sys
import pandas as pd
import json
import os
import matplotlib.pyplot as plt  # <-- Was missing from the bottom
from collections import defaultdict, namedtuple
from tabulate import tabulate

Mounted at /content/drive


In [ ]:
# Helper Functions

# A simple edge representation
Edge = namedtuple('Edge', ['u', 'v', 'weight'])

class Graph:
    """
    A simple class for a weighted, undirected graph.
    """
    def __init__(self):
        # adj: {u: {v: weight}}
        self.adj = defaultdict(dict)
        self.vertices = set()

    def add_edge(self, u, v, weight):
        """Adds an edge to the graph."""
        self.adj[u][v] = weight
        self.adj[v][u] = weight
        self.vertices.add(u)
        self.vertices.add(v)

    def get_edges(self):
        """Returns a list of all unique edges."""
        edges_set = set()
        for u, neighbors in self.adj.items():
            for v, weight in neighbors.items():
                # Sort the vertices in the edge tuple to make it unique
                # (u, v) and (v, u) will both become (min(u,v), max(u,v))
                edge_tuple = tuple(sorted((u, v)))

                # Add the unique edge tuple with its weight
                edges_set.add((edge_tuple[0], edge_tuple[1], weight))

        # Convert the set of tuples back to Edge objects
        return [Edge(u, v, w) for u, v, w in edges_set]

    def get_neighbors(self, u):
        """Returns neighbors of a node as (neighbor, weight) tuples."""
        return self.adj.get(u, {}).items()

    def get_degree(self, u):
        """Returns the degree of a node."""
        return len(self.adj.get(u, {}))

    def get_weight(self, u, v):
        """Gets the weight of an edge."""
        return self.adj.get(u, {}).get(v, float('inf'))

    def remove_vertex(self, u):
        """Removes a vertex and all its incident edges."""
        if u not in self.vertices:
            return

        neighbors = list(self.adj.get(u, {}).keys())
        for v in neighbors:
            if v in self.adj:
                del self.adj[v][u]

        del self.adj[u]
        self.vertices.remove(u)

    def calculate_cost(self):
        """Calculates the total cost of all edges in the graph."""
        cost = 0
        for u, v, weight in self.get_edges():
            cost += weight
        return cost

    def __str__(self):
        return f"Graph with {len(self.vertices)} vertices and {len(self.get_edges())} edges."


class UnionFind:
    """A helper class for Kruskal's algorithm (Disjoint Set Union)."""
    def __init__(self, vertices):
        self.parent = {v: v for v in vertices}
        self.rank = {v: 0 for v in vertices}

    def find(self, i):
        """Finds the root of the set containing i (with path compression)."""
        if self.parent[i] == i:
            return i
        self.parent[i] = self.find(self.parent[i])
        return self.parent[i]

    def union(self, i, j):
        """Merges the sets containing i and j (by rank)."""
        root_i = self.find(i)
        root_j = self.find(j)
        if root_i != root_j:
            if self.rank[root_i] < self.rank[root_j]:
                self.parent[root_i] = root_j
            elif self.rank[root_i] > self.rank[root_j]:
                self.parent[root_j] = root_i
            else:
                self.parent[root_j] = root_i
                self.rank[root_i] += 1
            return True
        return False

def kruskal_mst(graph):
    """
    Computes the Minimum Spanning Tree (MST) of a graph using Kruskal's.
    Returns a new Graph and the total cost.
    """
    mst = Graph()
    cost = 0
    edges = sorted(graph.get_edges(), key=lambda e: e.weight)
    uf = UnionFind(graph.vertices)

    for u, v, weight in edges:
        if uf.union(u, v):
            mst.add_edge(u, v, weight)
            cost += weight

    return mst, cost

def dijkstra_multi_source(graph, sources):
    """
    Runs Dijkstra's from a set of source nodes.
    Returns (distances, predecessors) maps.
    """
    dist = {v: float('inf') for v in graph.vertices}
    prev = {v: None for v in graph.vertices}
    pq = [] # Priority queue: (distance, vertex)

    for s in sources:
        if s in graph.vertices:
            dist[s] = 0
            heapq.heappush(pq, (0, s))

    while pq:
        d, u = heapq.heappop(pq)

        if d > dist[u]:
            continue

        for v, weight in graph.get_neighbors(u):
            new_dist = d + weight
            if new_dist < dist[v]:
                dist[v] = new_dist
                prev[v] = u
                heapq.heappush(pq, (new_dist, v))

    return dist, prev

In [ ]:
# Brute Force Algorithm

def create_induced_subgraph(original_graph, nodes):
    """Creates a subgraph containing only the specified nodes and their edges."""
    subgraph = Graph()
    nodes = set(nodes)

    # Add nodes first (in case some are isolated)
    for v in nodes:
        subgraph.vertices.add(v)

    for u, v, weight in original_graph.get_edges():
        if u in nodes and v in nodes:
            subgraph.add_edge(u, v, weight)
    return subgraph

def solve_brute_force(graph, terminals):
    """
    Solves the Steiner Tree problem using the exact Brute Force algorithm.
    """

    # bestCost <- infinity
    best_cost = float('inf')
    # bestTree <- empty
    best_tree = Graph()

    # S <- V \ T (Possible Steiner nodes)
    terminals = set(terminals)
    potential_steiners = list(graph.vertices - terminals)

    # for each subset X ⊆ S do
    for i in range(len(potential_steiners) + 1):
        for subset_X in itertools.combinations(potential_steiners, i):

            # V' <- T U X
            V_prime = terminals.union(set(subset_X))

            # G' <- induced subgraph of G on V'
            G_prime = create_induced_subgraph(graph, V_prime)

            # if G' is connected then
            # MST <- MinimumSpanningTree(G')
            mst, cost = kruskal_mst(G_prime)

            # A valid MST must span all nodes in V'
            is_connected = len(mst.vertices) == len(V_prime) and \
                           len(mst.get_edges()) == len(V_prime) - 1

            if len(V_prime) == 1 and len(mst.vertices) == 1:
                is_connected = True

            if is_connected:
                # cost -> total weight of MST

                # if cost < bestCost then
                if cost < best_cost:
                    # bestCost <- cost
                    # bestTree <- MST
                    best_cost = cost
                    best_tree = mst

    # return bestTree, bestCost
    return best_tree, best_cost

In [ ]:
# Approximation Algorithm

def solve_takahashi_approximation(graph, terminals):
    """
    Solves the Steiner Tree problem using the Takahashi & Matsuyama
    approximation algorithm.
    """

    solution_tree = Graph()
    terminals = set(terminals)

    if not terminals:
        return solution_tree, 0

    # Select any terminal t0 ∈ T
    t0 = terminals.pop() # Removes and returns an arbitrary element

    # TreeVertices <- {t0}
    tree_vertices = {t0}
    solution_tree.vertices.add(t0)

    # RemainingTerminals <- T \ {t0} (terminals is already modified)
    remaining_terminals = terminals

    # while RemainingTerminals is not empty do
    while remaining_terminals:

        # Find terminal t ∈ RemainingTerminals and vertex v ∈ TreeVertices
        # such that the shortest path distance d(v, t) is minimized.
        dist, prev = dijkstra_multi_source(graph, tree_vertices)

        closest_terminal = None
        min_path_cost = float('inf')

        for t in remaining_terminals:
            if dist[t] < min_path_cost:
                min_path_cost = dist[t]
                closest_terminal = t

        if closest_terminal is None:
            break

        # Add all vertices and edges along the shortest path from v to t
        curr = closest_terminal
        while curr not in tree_vertices:
            pred = prev[curr]
            if pred is None:
                break

            weight = graph.get_weight(pred, curr)
            solution_tree.add_edge(pred, curr, weight)
            tree_vertices.add(curr)

            curr = pred

        # Remove t from RemainingTerminals
        remaining_terminals.remove(closest_terminal)

    # return (TreeVertices, TreeEdges)
    cost = solution_tree.calculate_cost()
    return solution_tree, cost

In [ ]:
# Heurestic Algorithm

def solve_heuristic(graph, terminals):
    """
    Solves the Steiner Tree problem using the MST-Pruning Heuristic.
    """

    terminals = set(terminals)

    # MST_G <- MinimumSpanningTree(G)
    mst_g, _ = kruskal_mst(graph)
    heuristic_tree = copy.deepcopy(mst_g)

    # Iteratively prune leaf nodes that are not terminals
    pruning_occurred = True

    while pruning_occurred:
        pruning_occurred = False

        # find all leaf nodes currently in MST_G
        nodes_to_prune = []

        # Iterate over list() to safely remove items from the set
        for v in list(heuristic_tree.vertices):

            # if v ∉ T and degree(v) == 1 then
            if v not in terminals and heuristic_tree.get_degree(v) == 1:
                nodes_to_prune.append(v)

        if nodes_to_prune:
            pruning_occurred = True
            for v in nodes_to_prune:
                # emove v and its incident edge from MST_G
                heuristic_tree.remove_vertex(v)

    # return MST_G
    cost = heuristic_tree.calculate_cost()
    return heuristic_tree, cost

In [ ]:
# Generate Data

def generate_test_case(num_vertices, num_edges, terminal_percentage):
    """
    Generates a random, connected graph and a set of terminals.
    """
    if num_vertices <= 0:
        return Graph(), set()

    # Ensure edge count is valid
    max_edges = num_vertices * (num_vertices - 1) // 2
    num_edges = min(num_edges, max_edges)
    num_edges = max(num_edges, num_vertices - 1) # Must be at least a tree

    graph = Graph()
    nodes = list(range(num_vertices))
    for v in nodes:
        graph.vertices.add(v)

    edges_added = 0
    added_edges_set = set()

    # Step 1: Guarantee connectivity by building a random tree
    visited = {0}
    unvisited = set(nodes[1:])

    while unvisited:
        u = random.choice(list(visited))
        v = random.choice(list(unvisited))

        weight = random.randint(1, 100)
        graph.add_edge(u, v, weight)
        added_edges_set.add(tuple(sorted((u, v))))

        visited.add(v)
        unvisited.remove(v)
        edges_added += 1

    # Step 2: Add remaining edges to reach the target count
    edges_to_add = num_edges - edges_added

    while edges_to_add > 0 and len(added_edges_set) < max_edges:
        u = random.choice(nodes)
        v = random.choice(nodes)

        edge_tuple = tuple(sorted((u, v)))

        if u != v and edge_tuple not in added_edges_set:
            weight = random.randint(1, 100)
            graph.add_edge(u, v, weight)
            added_edges_set.add(edge_tuple)
            edges_to_add -= 1

    # Step 3: Select terminals
    num_terminals = int(num_vertices * (terminal_percentage / 100.0))
    num_terminals = max(2, min(num_terminals, num_vertices)) # Ensure 2 <= k <= n

    terminals = set(random.sample(nodes, num_terminals))

    return graph, terminals


def save_graph_to_file(graph, terminals, filename):
    """Saves the graph and terminals to a JSON file."""
    # Get actual graph properties
    v = len(graph.vertices)
    edges_list = [[e.u, e.v, e.weight] for e in graph.get_edges()]
    e = len(edges_list)
    k = len(terminals)

    data = {
        "v": v, "e": e, "k": k,
        "vertices": list(graph.vertices),
        "edges": edges_list,
        "terminals": list(terminals)
    }

    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(filename), exist_ok=True)

    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

def load_graph_from_file(filename):
    """Loads a graph and terminals from a JSON file."""
    with open(filename, 'r') as f:
        data = json.load(f)

    graph = Graph()

    # Add all vertices first (important for nodes that might be isolated)
    for v in data['vertices']:
        graph.vertices.add(v)

    # Add all edges
    for u, v, w in data['edges']:
        graph.add_edge(u, v, w)

    terminals = set(data['terminals'])

    # Return the graph and its properties
    return graph, terminals, data['v'], data['e'], data['k']

In [ ]:
# Experiment Runner

def run_experiment_set(config):
    """
    Runs a set of experiments, collects data into a pandas DataFrame,
    prints a formatted table, and saves to CSV.
    """
    print("\n" + "="*110)
    print(f"--- Running Experiment: {config['name']} Graphs ---")
    print(f"--- {config['runs']} instances, Algos: {config['algos']} ---")
    print("="*110)

    results_data = []

    # Define headers
    headers = ["Instance", "V", "E", "k"]
    if 'bf' in config['algos']:
        headers.extend(["Brute Cost", "Brute Time (ms)"])
    if 'app' in config['algos']:
        headers.extend(["Approx Cost", "Approx Time (ms)"])
    if 'heu' in config['algos']:
        headers.extend(["Heuristic Cost", "Heuristic Time (ms)"])
    if 'bf' in config['algos'] and 'app' in config['algos']:
        headers.append("Approx/Opt")
    if 'bf' in config['algos'] and 'heu' in config['algos']:
        headers.append("Heuristic/Opt")
    if 'app' in config['algos'] and 'heu' in config['algos']:
        headers.append("Heuristic/Approx")

    for i in range(config['runs']):
        instance_label = f"Test {i+1}"

        # Define graph filename and check if it exists
        graph_dir = os.path.join(DRIVE_ROOT_PATH, "test_graphs")
        graph_filename = os.path.join(graph_dir, f"graph_{config['name'].lower()}_{i+1}.json")

        # Check if file exists and we are NOT forcing regeneration
        if os.path.exists(graph_filename) and not config.get("force_regenerate", False):
            print(f"Loading graph for {instance_label} from {graph_filename}...")
            graph, terminals, v, e, k = load_graph_from_file(graph_filename)

        else:
            # 1. Generate random parameters
            print(f"Generating new graph for {instance_label} and saving to {graph_filename}...")
            v_target = random.randint(*config['v_range'])
            e_target = random.randint(*config['e_range'])
            t_perc = random.randint(*config['t_percent'])

            # 2. Generate the test case
            graph, terminals = generate_test_case(v_target, e_target, t_perc)

            # 3. Save the new graph
            save_graph_to_file(graph, terminals, graph_filename)

            # 4. Get the *actual* properties from the generated graph
            v = len(graph.vertices)
            e = len(graph.get_edges())
            k = len(terminals)

        row = {
            "Instance": instance_label,
            "V": v,
            "E": e,
            "k": k
        }

        # 3. Run Algorithms
        brute_cost = None
        approx_cost = None
        if 'bf' in config['algos']:
            start_time = time.perf_counter()
            _, cost = solve_brute_force(graph, terminals)
            elapsed_ms = (time.perf_counter() - start_time) * 1000
            row["Brute Cost"] = cost
            row["Brute Time (ms)"] = elapsed_ms
            brute_cost = cost # For ratio calculation

        if 'app' in config['algos']:
            start_time = time.perf_counter()
            _, cost = solve_takahashi_approximation(graph, terminals)
            elapsed_ms = (time.perf_counter() - start_time) * 1000
            row["Approx Cost"] = cost
            row["Approx Time (ms)"] = elapsed_ms
            approx_cost = cost
            if brute_cost is not None and brute_cost > 0:
                row["Approx/Opt"] = cost / brute_cost
            elif 'bf' in config['algos']:
                row["Approx/Opt"] = 1.0 if cost == brute_cost else float('nan')

        if 'heu' in config['algos']:
            start_time = time.perf_counter()
            _, cost = solve_heuristic(graph, terminals)
            elapsed_ms = (time.perf_counter() - start_time) * 1000
            row["Heuristic Cost"] = cost
            row["Heuristic Time (ms)"] = elapsed_ms
            if brute_cost is not None and brute_cost > 0:
                row["Heuristic/Opt"] = cost / brute_cost
            elif 'bf' in config['algos']:
                 row["Heuristic/Opt"] = 1.0 if cost == brute_cost else float('nan')

            if approx_cost is not None and approx_cost > 0:
                row["Heuristic/Approx"] = cost / approx_cost
            elif 'app' in config['algos']:
                 row["Heuristic/Approx"] = 1.0 if cost == approx_cost else float('nan')

        results_data.append(row)

    # 4. Process and Display Results
    if not results_data:
        print("No results to display.")
        return

    df = pd.DataFrame(results_data)
    final_headers = [h for h in headers if h in df.columns]
    df = df[final_headers]

    print(tabulate(df, headers='keys', tablefmt='grid', floatfmt=".4f", showindex=False))

    results_dir = os.path.join(DRIVE_ROOT_PATH, "results")
    os.makedirs(results_dir, exist_ok=True)

    # Save to the 'results' subfolder
    csv_filename = os.path.join(results_dir, f"results_{config['name'].lower()}.csv")
    df.to_csv(csv_filename, index=False, float_format="%.4f")
    print(f"\nResults saved to {csv_filename}")


def main():
    """
    Defines and runs the experimental setups from the report.
    """

    # Set this to True if you want to generate new graphs,
    # False if you want to re-use existing ones.
    FORCE_REGEN = False

    # Config from Page 6 of steiner-tree-problem-report.pdf
    SMALL_CONFIG = {
        "name": "Small",
        "v_range": (10, 20), # 10-20 V, keeping it on the lower side for BF
        "e_range": (15, 50), # 15-50 E
        "t_percent": (10, 20),
        "runs": 5, # 5-10 runs
        "algos": ['bf', 'app', 'heu'],
        "force_regenerate": FORCE_REGEN
    }

    MEDIUM_CONFIG = {
        "name": "Medium",
        "v_range": (50, 200), # 50-200 V
        "e_range": (100, 1000), # 100-1000 E
        "t_percent": (10, 20),
        "runs": 5, # 5-10 runs
        "algos": ['app', 'heu'],
        "force_regenerate": FORCE_REGEN
    }

    LARGE_CONFIG = {
        "name": "Large",
        "v_range": (500, 2000), # 500-2000 V
        "e_range": (2000, 10000), # 2000-10000 E
        "t_percent": (10, 20),
        "runs": 5, # 3-5 runs
        "algos": ['app', 'heu'],
        "force_regenerate": FORCE_REGEN
    }

    # Run the experiments
    run_experiment_set(SMALL_CONFIG)
    run_experiment_set(MEDIUM_CONFIG)
    run_experiment_set(LARGE_CONFIG)

In [ ]:
# Plot Graphs

def generate_plots():
    """
    Reads the CSV output files from Google Drive and generates plots to visualize
    runtime and solution quality.
    """
    print("\n" + "="*80)
    print("--- Generating Plots ---")
    print("="*80)

    results_dir = os.path.join(DRIVE_ROOT_PATH, "results")
    plots_dir = os.path.join(DRIVE_ROOT_PATH, "plots")
    os.makedirs(plots_dir, exist_ok=True)

    # Define CSV Paths (now in 'results' subfolder)
    csv_small = os.path.join(results_dir, "results_small.csv")
    csv_medium = os.path.join(results_dir, "results_medium.csv")
    csv_large = os.path.join(results_dir, "results_large.csv")

    # Plot 1: Runtime vs. Graph Size (Log Scale Line Plot)
    try:
        df_all = []

        if os.path.exists(csv_small):
            df_all.append(pd.read_csv(csv_small))
        if os.path.exists(csv_medium):
            df_all.append(pd.read_csv(csv_medium))
        if os.path.exists(csv_large):
            df_all.append(pd.read_csv(csv_large))

        if not df_all:
            print("No CSV files found. Skipping Plot 1.")
            raise FileNotFoundError

        df_all = pd.concat(df_all).sort_values(by="V")

        plt.figure(figsize=(10, 6))

        if "Brute Time (ms)" in df_all.columns:
            df_brute = df_all.dropna(subset=["Brute Time (ms)"])
            plt.plot(df_brute["V"], df_brute["Brute Time (ms)"], marker='o', linestyle='--', label="Brute Force")

        if "Approx Time (ms)" in df_all.columns:
            plt.plot(df_all["V"], df_all["Approx Time (ms)"], marker='s', label="Approx (T&M)")

        if "Heuristic Time (ms)" in df_all.columns:
            plt.plot(df_all["V"], df_all["Heuristic Time (ms)"], marker='^', label="Heuristic (MST Pruning)")

        plt.yscale('log')
        plt.xlabel("Number of Vertices (V)")
        plt.ylabel("Time (ms) - Log Scale")
        plt.title("Algorithm Runtime vs. Graph Size")
        plt.legend()
        plt.grid(True, which="both", ls="--", alpha=0.5)

        plot1_filename = os.path.join(plots_dir, "plot_1_runtime_vs_size.png")
        plt.savefig(plot1_filename, dpi=150, bbox_inches="tight")
        print(f"Saved Plot 1: {plot1_filename}")
        plt.close()

    except FileNotFoundError:
        print("Could not generate Plot 1: Missing one or more results CSVs.")
    except Exception as e:
        print(f"An error occurred in Plot 1: {e}")


    # Plot 2: Solution Quality (Grouped Bar Chart for Small Graphs)
    try:
        if not os.path.exists(csv_small):
            print(f"No '{csv_small}' found. Skipping Plot 2.")
            raise FileNotFoundError

        df_small = pd.read_csv(csv_small)

        labels = df_small["Instance"]
        x = list(range(len(labels)))
        width = 0.25

        fig, ax = plt.subplots(figsize=(12, 7))

        rects1 = ax.bar([pos - width for pos in x], df_small["Brute Cost"], width, label="Brute (Optimal)", color='#4CAF50')
        rects2 = ax.bar(x, df_small["Approx Cost"], width, label="Approx (T&M)", color='#2196F3')
        rects3 = ax.bar([pos + width for pos in x], df_small["Heuristic Cost"], width, label="Heuristic", color='#F44336')

        ax.set_ylabel("Solution Cost")
        ax.set_xlabel("Test Instance")
        ax.set_title("Solution Quality (Cost) on Small Graphs")
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend()
        ax.grid(True, axis='y', ls='--', alpha=0.5)

        def add_labels_manual(rects):
            for rect in rects:
                height = rect.get_height()
                ax.annotate(f'{height:.0f}',
                            xy=(rect.get_x() + rect.get_width() / 2, height),
                            xytext=(0, 3),
                            textcoords="offset points",
                            ha='center', va='bottom')

        add_labels_manual(rects1)
        add_labels_manual(rects2)
        add_labels_manual(rects3)

        fig.tight_layout()

        plot2_filename = os.path.join(plots_dir, "plot_2_solution_quality_small.png")
        plt.savefig(plot2_filename, dpi=150, bbox_inches="tight")
        print(f"Saved Plot 2: {plot2_filename}")
        plt.close()

    except FileNotFoundError:
        pass
    except Exception as e:
        print(f"An error occurred in Plot 2: {e}")


    # Plot 3: Approx Cost vs. Heuristic Cost (Scatter Plot)
    try:
        df_med_large = []
        if os.path.exists(csv_medium):
            df_med_large.append(pd.read_csv(csv_medium))
        if os.path.exists(csv_large):
            df_med_large.append(pd.read_csv(csv_large))

        if not df_med_large:
            print("No Medium/Large CSVs found. Skipping Plot 3.")
            raise FileNotFoundError

        df_med_large = pd.concat(df_med_large)

        plt.figure(figsize=(8, 8))
        plt.scatter(df_med_large["Approx Cost"], df_med_large["Heuristic Cost"], alpha=0.7)

        max_val = max(df_med_large["Approx Cost"].max(), df_med_large["Heuristic Cost"].max())
        min_val = min(df_med_large["Approx Cost"].min(), df_med_large["Heuristic Cost"].min())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', label="y = x (Equal Cost)")

        plt.xlabel("Approx (T&M) Cost")
        plt.ylabel("Heuristic (MST Pruning) Cost")
        plt.title("Cost Comparison (Medium & Large Graphs)")
        plt.legend()
        plt.grid(True, ls="--", alpha=0.5)
        plt.gca().set_aspect('equal', 'box')

        plot3_filename = os.path.join(plots_dir, "plot_3_cost_comparison_scatter.png")
        plt.savefig(plot3_filename, dpi=150, bbox_inches="tight")
        print(f"Saved Plot 3: {plot3_filename}")
        plt.close()

    except FileNotFoundError:
        pass
    except Exception as e:
        print(f"An error occurred in Plot 3: {e}")

    print("\n--- Plot generation complete. ---")

In [ ]:
# run the experiments to generate the CSV files
# Comment this line out if you just want to re-run plotting on existing CSVs
main()

# run the plotting function to analyze the CSV files
generate_plots()


--- Running Experiment: Small Graphs ---
--- 5 instances, Algos: ['bf', 'app', 'heu'] ---
Loading graph for Test 1 from /content/drive/My Drive/Colab Notebooks/bpta/stp/test_graphs/graph_small_1.json...
Loading graph for Test 2 from /content/drive/My Drive/Colab Notebooks/bpta/stp/test_graphs/graph_small_2.json...
Loading graph for Test 3 from /content/drive/My Drive/Colab Notebooks/bpta/stp/test_graphs/graph_small_3.json...
Loading graph for Test 4 from /content/drive/My Drive/Colab Notebooks/bpta/stp/test_graphs/graph_small_4.json...
Loading graph for Test 5 from /content/drive/My Drive/Colab Notebooks/bpta/stp/test_graphs/graph_small_5.json...
+------------+-----+-----+-----+--------------+-------------------+---------------+--------------------+------------------+-----------------------+--------------+-----------------+--------------------+
| Instance   |   V |   E |   k |   Brute Cost |   Brute Time (ms) |   Approx Cost |   Approx Time (ms) |   Heuristic Cost |   Heuristic Time (